In [1]:
import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv
load_dotenv()

## load the GROQ And OpenAI API KEY 
groq_api_key=os.getenv('GROQ_API_KEY')
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")


llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [5]:
response=llm.invoke("Tell me a joke")
response

AIMessage(content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!", response_metadata={'token_usage': {'completion_time': 0.013659397, 'completion_tokens': 18, 'prompt_time': 0.003361263, 'prompt_tokens': 14, 'queue_time': None, 'total_time': 0.01702066, 'total_tokens': 32}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-72023fb5-f9c0-4c52-b0c1-17d0048a8473-0')

In [6]:
response=llm.invoke("What are the components of soil?")

lines=response.content.split("\n")
for line in lines:
    print(line)

Soil is a complex ecosystem that consists of a mixture of mineral and organic components, as well as living organisms. The main components of soil are:

1. **Mineral particles**: These are the inorganic particles that give soil its structure and texture. They are the result of weathered rocks and minerals, such as:
	* Silica (SiO2) - quartz, flint, and sand
	* Alumina (Al2O3) - clay, kaolin, and bauxite
	* Iron oxides (Fe2O3) - iron oxide, hematite, and magnetite
	* Calcium carbonate (CaCO3) - limestone and dolomite
2. **Organic matter**: This is the fraction of the soil that is composed of decomposed plant and animal residues, such as:
	* Humus - the dark, carbon-rich substance formed from decomposed plant debris
	* Fulvic acids - complex organic compounds that contribute to soil fertility
	* Humic acids - complex organic compounds that play a role in soil structure and fertility
3. **Water**: Soil's water content is crucial for plant growth and can range from 0.1 to 1000 g/kg. Water 

# Prompt template

In [4]:
from langchain.prompts import PromptTemplate

noinputprompt=PromptTemplate(input_variables=[],
                            template="Tell me a python trick")
print(noinputprompt)

input_variables=[] template='Tell me a python trick'


In [46]:
formattednoinputprompt=noinputprompt.format()
response=llm.invoke(formattednoinputprompt)
lines=response.content.split("\n")
for line in lines:
    print(line)

Here's a Python trick:

**Using a dictionary to reverse a list**

In Python, you can use a dictionary to reverse a list in a very concise way:
```python
my_list = [1, 2, 3, 4, 5]
reversed_list = dict(enumerate(reversed(my_list))).values()
print(list(reversed_list))  # [5, 4, 3, 2, 1]
```
This works because the `enumerate` function returns an iterator that produces tuples containing the index and value of each element in the list. The `reversed` function reverses the order of the tuples. The `dict` function then creates a dictionary from the reversed tuples, using the indices as keys and the values as values. Finally, the `values` method returns an iterator over the values, which we convert back to a list using the `list` function.

This trick is a great example of how Python's dynamic typing and built-in functions can be used in creative ways to solve problems.


In [6]:
noinputprompt=PromptTemplate.from_template(template="Tell me a python trick")
print(noinputprompt)

input_variables=[] template='Tell me a python trick'


### Single input prompt template

In [48]:
oneinput=PromptTemplate(input_variables=["languages"],
                        template="Tell me a {language} trick")
oneinput

PromptTemplate(input_variables=['language'], template='Tell me a {language} trick')

In [49]:
formatoneinput=oneinput.format(language="SQL")
response=llm.invoke(formatoneinput)
lines=response.content.split("\n")
for line in lines:
    print(line)

Here's a SQL trick:

**Using a subquery as a derived table in the FROM clause**

This trick allows you to use a subquery as a virtual table in the FROM clause, which can simplify complex queries and improve performance.

Suppose you want to find the top 3 departments with the highest average salary, along with their average salary and total salary. You can use the following query:
```sql
WITH department_avg_salary AS (
  SELECT department, AVG(salary) AS avg_salary, SUM(salary) AS total_salary
  FROM employees
  GROUP BY department
)
SELECT dept, avg_salary, total_salary
FROM department_avg_salary
ORDER BY avg_salary DESC
LIMIT 3;
```
In this query, the subquery `department_avg_salary` is used as a derived table in the FROM clause. The subquery calculates the average and total salary for each department, and the outer query selects the top 3 departments with the highest average salary.

This trick is useful when you need to perform complex calculations or aggregations, and you want to 

In [50]:
oneinput=PromptTemplate.from_template("Tell me a the capital of the {country}")
formattedoneinput=oneinput.format(country="Russia")
response=llm.invoke(formattedoneinput)
lines=response.content.split("\n")
for line in lines:
    print(line)

The capital of Russia is Moscow!


### Multiple input prompt template

In [51]:
multiinput=PromptTemplate(input_variables=["country","anything"],template="Can you tell me about the {country} and its famous {anything}")
formattedmulti=multiinput.format(country="India",anything="cities")
response=llm.invoke(formattedmulti)
lines=response.content.split("\n")
for line in lines:
    print(line)

India is a vast and diverse country with a rich history, culture, and architecture. Here are some of the most famous cities in India:

1. **New Delhi**: The capital city of India, New Delhi is known for its iconic landmarks like the Red Fort, Qutub Minar, and India Gate. It's a bustling metropolis with a mix of old and new architecture.
2. **Mumbai**: Formerly known as Bombay, Mumbai is India's financial capital and entertainment hub. It's famous for its Bollywood film industry, beaches like Juhu and Versova, and the iconic Gateway of India monument.
3. **Bangalore**: Known as the "Silicon Valley of India," Bangalore is a hub for the IT industry. It's also famous for its gardens, lakes, and the famous Lalbagh Botanical Garden.
4. **Chennai**: The capital of Tamil Nadu state, Chennai is known for its rich cultural heritage, temples like the Kapaleeswarar Temple, and the famous Marina Beach.
5. **Hyderabad**: This city in Telangana state is famous for its pearl industry, historic landmar

In [52]:
multiinput=PromptTemplate.from_template("Tell me a {language}{topic} trick")
formattedmulti=multiinput.format(language="python",topic="array")
response=llm.invoke(formattedmulti)
lines=response.content.split("\n")
for line in lines:
    print(line)

Here's a Python array trick:

**Using the `map` function to apply a lambda function to each element of an array**

Imagine you have an array of numbers, and you want to square each number. You could do this using a loop, but that's tedious. With Python, you can use the `map` function and a lambda function to achieve the same result in a single line of code:
```python
import array

numbers = [1, 2, 3, 4, 5]

squared_numbers = list(map(lambda x: x**2, numbers))

print(squared_numbers)  # [1, 4, 9, 16, 25]
```
The `map` function applies the lambda function (`lambda x: x**2`) to each element of the `numbers` array, and returns a new array with the results.

This trick works with any iterable (not just arrays), and with any function that takes one argument. For example, you can use it to convert all strings in an array to uppercase:
```python
strings = ['hello', 'world', 'python']
uppercase_strings = list(map(lambda s: s.upper(), strings))
print(uppercase_strings)  # ['HELLO', 'WORLD', 'PYT

# Chat Models Prompt Template

In [3]:
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,PromptTemplate

In [3]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that translates {input_language} to {output_language}"),
    ("human","{text}")
])

formatchatprompt=chatprompt.format_messages(
    input_language="English",
    output_language="Kannada",
    text="Hello, how are you nice to meet you"
)

print(formatchatprompt)

[SystemMessage(content='You are a helpful assistant that translates English to Kannada'), HumanMessage(content='Hello, how are you nice to meet you')]


In [27]:
res=llm.invoke(formatchatprompt)
res.content

"ನಿಮ್ಮನ್ನು ಭೇಟಿ ಮಾಡಿದ್ದೇನೆ ಸ್ವಾಗತ! Namaste! (It's nice to meet you!)"

In [37]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","You are a creative and funny chat assistant that provides captions for social media posts based on three words that describe the post {word1},{word2},{word3}"),
])

formatchatprompt=chatprompt.format_messages(
    word1="happy",
    word2="twinning",
    word3="beach",
) 

res=llm.invoke(formatchatprompt)
lines=res.content.split('\n')
for line in lines:
    print(line)

"Sand, Sun, and Sisterly Love! Twins' Beach Day #Twinning #BeachVibes #SisterGoals"


In [41]:
chatprompt=ChatPromptTemplate.from_messages([
    ("system","Imagine a witty and inventive chat assistant that crafts social media captions tailored to posts, using three descriptive words: {word1}, {word2}, and {word3}."),
])

formatchatprompt=chatprompt.format_messages(
    word1="happy",
    word2="twinning",
    word3="beach",
) 

res=llm.invoke(formatchatprompt)
lines=res.content.split('\n')
for line in lines:
    print(line)

Let's get this caption party started!

Since you've given me three fantastic descriptive words - happy, twinning, and beach - I'll weave them into some fabulous social media captions for you! Here are a few options:

1. "Sunkissed and feeling happy! Nothing beats a beach day with my twins! #beachlife #happyvibes #twinning"
2. "Twinning on the beach, because happiness is catching! #happyplace #beachday #twinning"
3. "Surf's up, and so are our smiles! Happy beach day with my twin, because twinning is winning! #beachlife #happy #twinning"
4. "When in doubt, head to the beach! Happy vibes only with my twin by my side. #beachvibes #happy #twinning"
5. "Twinning and winning on the beach! Happy to share this sun-kissed adventure with my twin. #beachday #happy #twinning"

Feel free to pick the one that makes you go "Ah-ha!" or mix and match to create your own unique caption!


In [4]:
template="Picture yourself as a clever and creative chat assistant that crafts perfect social media captions using three descriptive words: {word1}, {word2}, and {word3}."

prompt=PromptTemplate(input_variables=["word1","word2","word3"],
                    template=template)

formattedprompt=prompt.format(word1="sunny",word2="riverside",word3="friends")
response=llm.invoke(formattedprompt)
lines=response.content.split('\n')
for line in lines:
    print(line)

What a lovely prompt! Here are some creative social media captions using the words "sunny", "riverside", and "friends":

**Happy Moments**

1. "Sunny days with friends by the riverside, life is good!"
2. "Laughter and memories made with friends on a sunny riverside afternoon"

**Adventure Time**

1. "Exploring the great outdoors with friends on a sunny riverside escapade"
2. "Riverside adventures with friends under a sunny sky"

**Relaxation Mode**

1. "Sunny vibes and riverside tranquility with friends by my side"
2. "A sunny day spent with friends by the riverside, recharging and relaxing"

**Special Occasions**

1. "Celebrating [special occasion] with friends on a sunny riverside day"
2. "Surrounded by friends and sunshine on a riverside [special occasion] to remember"

**Inspirational Quotes**

1. "Just like the riverside, life is more beautiful with friends by your side. Sunny days ahead!"
2. "With friends by your side, every day can be a sunny riverside adventure"

I hope you enj